In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from collections import defaultdict
from gensim import corpora

In [3]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [4]:
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

In [5]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [6]:
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [7]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2021-02-17 05:26:17,539 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-17 05:26:17,541 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [8]:
from gensim import models

In [9]:
tfidf = models.TfidfModel(corpus)

2021-02-17 05:26:22,195 : INFO : collecting document frequencies
2021-02-17 05:26:22,197 : INFO : PROGRESS: processing document #0
2021-02-17 05:26:22,199 : INFO : calculating IDF weights for 9 documents and 12 features (28 matrix non-zeros)


In [12]:
doc_bow = [(0,1),(1,1)]

In [13]:
print(tfidf[doc_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [15]:
corpus_tfidf = tfidf[corpus]

In [16]:
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [17]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation

2021-02-17 05:39:41,092 : INFO : using serial LSI version on this node
2021-02-17 05:39:41,096 : INFO : updating model with new documents
2021-02-17 05:39:41,098 : INFO : preparing a new chunk of documents
2021-02-17 05:39:41,103 : INFO : using 100 extra samples and 2 power iterations
2021-02-17 05:39:41,104 : INFO : 1st phase: constructing (12, 102) action matrix
2021-02-17 05:39:41,112 : INFO : orthonormalizing (12, 102) action matrix
2021-02-17 05:39:41,124 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2021-02-17 05:39:41,131 : INFO : computing the final decomposition
2021-02-17 05:39:41,134 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2021-02-17 05:39:41,136 : INFO : processed documents up to #9
2021-02-17 05:39:41,140 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2021-02-17 05:39:41,143 : INFO : topic #

In [18]:
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [19]:
lsi_model.print_topics(2)

2021-02-17 05:41:22,568 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2021-02-17 05:41:22,571 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [20]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090323), (1, -0.520070330636184)] Human machine interface for lab abc computer applications
[(0, 0.1966759285914236), (1, -0.7609563167700051)] A survey of user opinion of computer system response time
[(0, 0.08992639972446352), (1, -0.7241860626752508)] The EPS user interface management system
[(0, 0.07585847652178093), (1, -0.6320551586003424)] System and human system engineering testing of EPS
[(0, 0.10150299184980004), (1, -0.5737308483002961)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378316), (1, 0.16115180214025696)] The generation of random binary unordered trees
[(0, 0.8774787673119835), (1, 0.16758906864659287)] The intersection graph of paths in trees
[(0, 0.9098624686818578), (1, 0.14086553628718884)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569276), (1, -0.05392907566389467)] Graph minors A survey


In [21]:
import os
import tempfile

In [34]:
with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

2021-02-17 05:59:18,263 : INFO : saving Projection object under /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi.projection, separately None
2021-02-17 05:59:18,267 : INFO : saved /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi.projection
2021-02-17 05:59:18,268 : INFO : saving LsiModel object under /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi, separately None
2021-02-17 05:59:18,270 : INFO : not storing attribute projection
2021-02-17 05:59:18,271 : INFO : not storing attribute dispatcher
2021-02-17 05:59:18,274 : INFO : saved /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi


In [35]:
loaded_lsi_model = models.LsiModel.load(tmp.name)

2021-02-17 05:59:47,981 : INFO : loading LsiModel object from /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi
2021-02-17 05:59:47,985 : INFO : loading id2word recursively from /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi.id2word.* with mmap=None
2021-02-17 05:59:47,987 : INFO : setting ignored attribute projection to None
2021-02-17 05:59:47,988 : INFO : setting ignored attribute dispatcher to None
2021-02-17 05:59:47,990 : INFO : loaded /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi
2021-02-17 05:59:47,991 : INFO : loading LsiModel object from /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi.projection
2021-02-17 05:59:47,994 : INFO : loaded /var/folders/vf/m4p5jj9s07x4v_df3mgg2fv40000gn/T/model-7zmhc_t0.lsi.projection


In [36]:
os.unlink(tmp.name)